Neural Point Process

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# dataset
x_train, y_train, p_train / data_loader
x_val, y_val, p_val = 
x_test, y_test, p_test = 


In [ ]:
def gaussian_kernel_matrix(X, Y, sigma):
    """
    Calculate the Gaussian kernel matrix between two sets of PyTorch tensors X and Y.

    Parameters:
    X (torch.Tensor): First set of tensors with shape (n, d), where n is the number of vectors and d is the dimensionality.
    Y (torch.Tensor): Second set of tensors with shape (m, d), where m is the number of vectors and d is the dimensionality.
    sigma (float): The kernel bandwidth parameter.

    Returns:
    torch.Tensor: The Gaussian kernel matrix of shape (n, m).
    """
    if X.size(1) != Y.size(1):
        raise ValueError("Input tensors must have the same dimension")

    n, m = X.size(0), Y.size(0)
    X = X.unsqueeze(1)  # Shape (n, 1, d)
    Y = Y.unsqueeze(0)  # Shape (1, m, d)

    diff = torch.norm(X - Y, dim=2)  # Pairwise Euclidean distances between vectors
    return torch.exp(- (diff ** 2) / (2 * sigma ** 2))


def pseudo_inverse(kernel_matrix, epsilon=1e-5):
    """
    Calculate the pseudo-inverse of a matrix using Singular Value Decomposition (SVD).

    Parameters:
    kernel_matrix (torch.Tensor): The matrix for which to compute the pseudo-inverse.
    epsilon (float): A small value to avoid division by zero.

    Returns:
    torch.Tensor: The pseudo-inverse of the input matrix.
    """
    U, S, V = torch.svd(kernel_matrix)
    S_inv = 1.0 / (S + epsilon)
    pseudo_inv = torch.mm(V, torch.mm(torch.diag(S_inv), U.t()))
    return pseudo_inv


class NPPLoss(nn.Module):
    def __init__(self, matrix):
        super(NPPLoss, self).__init__()
        self.matrix_list = matrix_list  # The matrix to be applied between MSE terms

    def forward(self, y_true, y_pred, pins):
        loss = 0
        # Calculate the mean squared error
        for i in range(y_true):
            # Apply the matrix operation to the MSE loss
            loss += torch.matmul((y_true[i] - y_pred[i][pins]).t(), torch.matmul(self.matrix_list[i], y_true[i] - y_pred[i][pins]))

        return loss
    


In [ ]:
# Define the encoder architecture
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Input: 1 channel (grayscale), Output: 32 channels
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Input: 32 channels, Output: 64 channels
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return x

# Define the decoder architecture
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),  # Input: 64 channels, Output: 32 channels
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=2, stride=2),  # Input: 32 channels, Output: 1 channel (grayscale)
            nn.Sigmoid(),  # Apply Sigmoid activation to constrain output in [0, 1]
        )
    
    def forward(self, x):
        x = self.decoder(x)
        return x

# Define the Autoencoder as a combination of Encoder and Decoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
# Hyperparameters
input_size = 28 * 28  # MNIST image size (28x28 pixels)
hidden_size = 64
batch_size = 128
learning_rate = 0.001
num_epochs = 20

# Load ??? dataset -- train data_loader
data_loader = torch.utils.data.DataLoader(dataset=???, batch_size=batch_size, shuffle=True)
# and also the test/val data_loader

# Create an instance of the Autoencoder
autoencoder = Autoencoder()

#calculate the pseudo_inv once before the training.

sigma = 1.0
matrix_list = []
for i in range(len(x_train)):
    # p_train is a n length list, where the i-th element contains L_i pairs of pins 
    X = Y = p_train[i] 
    kernel_matrix = gaussian_kernel_matrix(X, Y, sigma)  
    pseudo_inv_matrix = pseudo_inverse(kernel_matrix)
    matrix_list.append(pseudo_inv_matrix)


# Loss function and optimizer
criterion = NPPLoss(matrix_list)
baseline_criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for data in data_loader:
        x_train, y_train, p_train = data
        
        # Forward pass
        outputs = autoencoder(x_train)
        loss = criterion(y_train, outputs, p_train)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Visualize original and reconstructed images
with torch.no_grad():
    data_iter = iter(data_loader)
    images, _ = next(data_iter)
    images = images.view(images.size(0), -1)

    reconstructed_images = autoencoder(images)

    plt.figure(figsize=(12, 5))
    for i in range(5):
        # Original Images
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].view(28, 28).numpy(), cmap='gray')
        plt.title('Original')

        # Reconstructed Images
        plt.subplot(2, 5, i + 6)
        plt.imshow(reconstructed_images[i].view(28, 28).numpy(), cmap='gray')
        plt.title('Reconstructed')

    plt.show()

